<a href="https://colab.research.google.com/github/prithwis/AGI/blob/main/SnakeFrog_v4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![CC-BY-SA](https://licensebuttons.net/l/by-sa/3.0/88x31.png)<br>


![alt text](https://github.com/Praxis-QR/RDWH/raw/main/images/YantraJaalBanner.png)<br>




[Prithwis Mukerjee](http://www.linkedin.com/in/prithwis)<br>

#Evolving Snake-Frog Worlds with agents
for simulating Reinforcement Learning Applications

In [1]:
# 1. Update package list and install the NEW opengl names
!apt-get update > /dev/null 2>&1
!apt-get install -y xvfb ffmpeg freeglut3-dev python3-opengl libgl1-mesa-dev libglu1-mesa-dev mesa-utils > /dev/null 2>&1

# 2. Install the system dependencies (SWIG is the key here)
#!apt-get update
!apt-get install -y swig build-essential python3-dev > /dev/null 2>&1

# 3. Upgrade pip and setuptools to handle the build process better
!pip install --upgrade pip setuptools wheel > /dev/null 2>&1

# 4. Now install gymnasium with box2d support
!pip install "gymnasium[box2d]" > /dev/null 2>&1

# 5. Install the Python libraries
!pip install pyvirtualdisplay  pygame opencv-python > /dev/null 2>&1


In [2]:
import gymnasium as gym
try:
    env = gym.make("LunarLander-v3")
    print("Success! The physics engine is ready for AGI training.")
    env.close()
except Exception as e:
    print(f"Error: {e}")

/usr/local/lib/python3.12/dist-packages/pygame/pkgdata.py:25: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream, resource_exists


Success! The physics engine is ready for AGI training.


In [3]:
import pygame
import cv2
import numpy as np
import os

from IPython.display import Video, display


In [4]:
# --- 1. THE RECORDER CLASS ---
class VideoRecorder:
    def __init__(self, filename='simulation.avi', width=640, height=480, fps=30):
        self.filename = filename
        self.width = width
        self.height = height
        self.fps = fps
        self.fourcc = cv2.VideoWriter_fourcc(*'XVID')
        self.video_writer = None

    def start(self):
        if os.path.exists(self.filename): os.remove(self.filename)
        self.video_writer = cv2.VideoWriter(self.filename, self.fourcc, self.fps, (self.width, self.height))

    def record_frame(self, surface):
        # Convert Pygame surface to OpenCV format
        view = pygame.surfarray.array3d(surface)
        view = view.transpose([1, 0, 2])
        view = cv2.cvtColor(view, cv2.COLOR_RGB2BGR)
        view = cv2.resize(view, (self.width, self.height))
        self.video_writer.write(view)

    def stop(self):
        self.video_writer.release()
        # Convert to MP4 for browser compatibility
        output_mp4 = self.filename.replace('.avi', '.mp4')
        os.system(f"ffmpeg -y -i {self.filename} -c:v libx264 -pix_fmt yuv420p {output_mp4} -hide_banner -loglevel error")
        return output_mp4

In [5]:
# --- BASE CLASS: SIMPLE PHYSICS ---
class FrogSnakeWorldV1:
    def __init__(self, width=640, height=480):
        self.width = width
        self.height = height
        self.frog_pos = np.array([width/2, height/2])
        self.frog_vel = np.array([2.0, 2.0])
        # Simple snakes: just position and a fixed speed
        self.snakes = [{"pos": np.array([float(np.random.randint(0, width)),
                                        float(np.random.randint(0, height))]),
                        "speed": 3.0} for _ in range(3)]

    def update_frog(self, step_count):
        """Logic for the prey's movement."""
        if step_count % 15 == 0:
            angle = np.random.uniform(0, 2 * np.pi)
            self.frog_vel = np.array([np.cos(angle), np.sin(angle)]) * 5.0

        self.frog_pos += self.frog_vel

        # Wall Bouncing
        if self.frog_pos[0] <= 0 or self.frog_pos[0] >= self.width: self.frog_vel[0] *= -1
        if self.frog_pos[1] <= 0 or self.frog_pos[1] >= self.height: self.frog_vel[1] *= -1

    def update_snakes(self):
        """V1: Simple direct-tracking movement."""
        for s in self.snakes:
            direction = self.frog_pos - s["pos"]
            dist = np.linalg.norm(direction)
            if dist > 0:
                s["pos"] += (direction / dist) * s["speed"]

    def update(self, step_count):
        self.update_frog(step_count)
        self.update_snakes()

    def draw(self, surface):
        surface.fill((20, 20, 30))
        # Draw Frog (Green)
        pygame.draw.circle(surface, (0, 255, 150), (int(self.frog_pos[0]), int(self.frog_pos[1])), 12)
        # Draw Snakes (Red)
        for s in self.snakes:
            pygame.draw.rect(surface, (255, 80, 80), (int(s["pos"][0]-10), int(s["pos"][1]-10), 20, 20))


In [6]:
# --- DERIVED CLASS: COMPLEX PHYSICS ---
class FrogSnakeWorldV2(FrogSnakeWorldV1):
    def __init__(self, width=640, height=480):
        # Initialize V1 first
        super().__init__(width, height)

        # Add 'V2 specific' attributes: Velocity and Physics constants
        for s in self.snakes:
            s["vel"] = np.array([0.0, 0.0])
            s["accel_power"] = 0.7
            s["friction"] = 0.90 # Makes them 'slidier'

    def get_observation(self, snake_index):
        """The 'Sense' function for RL agents."""
        s = self.snakes[snake_index]
        # Only tell the snake where the frog is relative to itself
        relative_pos = self.frog_pos - s["pos"]
        return np.concatenate([relative_pos, s["vel"]])

    def update_snakes(self):
        """V2: Overrides V1 with Momentum-based movement."""
        for i, s in enumerate(self.snakes):
            obs = self.get_observation(i)
            rel_pos = obs[:2] # Extract relative distance from senses

            dist = np.linalg.norm(rel_pos)
            if dist > 0:
                # Apply acceleration toward target
                force = (rel_pos / dist) * s["accel_power"]
                s["vel"] += force

            # Physics: Friction slows them down, Velocity changes position
            s["vel"] *= s["friction"]
            s["pos"] += s["vel"]

In [7]:
def run_experiment(world_class, seconds=10):
    pygame.init()
    width, height = 640, 480
    surface = pygame.Surface((width, height))

    # Here is where the "Switch" happens:
    world = world_class(width, height)

    recorder = VideoRecorder(width=width, height=height)
    recorder.start()

    print(f"Running simulation using {world_class.__name__}...")

    for t in range(30 * seconds):
        world.update(t)
        world.draw(surface)
        recorder.record_frame(surface)

    final_file = recorder.stop()
    pygame.quit()
    display(Video(final_file, embed=True, width=600))



In [8]:
# --- CHOOSE YOUR WORLD HERE ---
# To run Simple:
run_experiment(FrogSnakeWorldV1)


Running simulation using FrogSnakeWorldV1...


In [13]:

# To run Complex with Momentum:
run_experiment(FrogSnakeWorldV2)

Running simulation using FrogSnakeWorldV2...


In [10]:
# Now Introducing the Random Agent

class RandomAgentV3:
    def __init__(self):
        # A random agent doesn't need to store much,
        # but a Neural Network agent would store its 'brain' here.
        pass

    def act(self, observation):
        """
        Takes in the state and returns a random action.
        0: Up, 1: Down, 2: Left, 3: Right
        """
        return np.random.randint(0, 4)

In [11]:
# Agent Driven World

class FrogSnakeWorldV3(FrogSnakeWorldV2):
    def __init__(self, width=640, height=480):
        super().__init__(width, height)
        # Link an agent to each snake
        self.agents = [RandomAgentV3() for _ in range(len(self.snakes))]

    def update_snakes(self):
        """V3: Now driven by external Agent actions instead of hardcoded chase."""
        for i, s in enumerate(self.snakes):
            # 1. Get the 'Sense' (Observation)
            obs = self.get_observation(i)

            # 2. Ask the Agent for an Action
            action = self.agents[i].act(obs)

            # 3. Translate Action (0-3) into Physics Force
            force = np.array([0.0, 0.0])
            power = s["accel_power"]

            if action == 0: force[1] = -power # Up
            elif action == 1: force[1] = power  # Down
            elif action == 2: force[0] = -power # Left
            elif action == 3: force[0] = power  # Right

            # 4. Apply Physics (Momentum/Friction inherited from V2)
            s["vel"] += force
            s["vel"] *= s["friction"]
            s["pos"] += s["vel"]

In [17]:
# Random "Dumb" snakes (The starting point for AGI)
run_experiment(FrogSnakeWorldV3)

Running simulation using FrogSnakeWorldV3...


In [18]:
# Introducing Reward

class FrogSnakeWorldV3R(FrogSnakeWorldV2):     # V3 with Reward
    def __init__(self, width=640, height=480):
        super().__init__(width, height)
        self.agents = [RandomAgentV3() for _ in range(len(self.snakes))]
        self.prev_dist = [0.0 for _ in range(len(self.snakes))]

    def get_reward(self, snake_index):
        s_pos = self.snakes[snake_index]["pos"]
        dist = np.linalg.norm(self.frog_pos - s_pos)

        # 1. Huge reward for catching the frog
        if dist < 20:
            return 100.0

        # 2. Dense reward: Did we get closer than last time?
        # This is called "Reward Shaping"
        reward = 0
        if dist < self.prev_dist[snake_index]:
            reward = 1.0  # Moving toward frog
        else:
            reward = -1.1 # Penalty for moving away or staying still

        self.prev_dist[snake_index] = dist
        return reward

    def update(self, step_count):
        """This overrides the V2 update to ensure rewards are passed back."""
        self.update_frog(step_count)
        step_reward = self.update_snakes()
        return step_reward # This returns a float to run_reward_experiment()

    def update_snakes(self):
        total_step_reward = 0

        for i, s in enumerate(self.snakes):
            obs = self.get_observation(i)
            action = self.agents[i].act(obs)

            # Physics (Same as before)
            force = np.array([0.0, 0.0])
            power = s["accel_power"]
            if action == 0: force[1] = -power
            elif action == 1: force[1] = power
            elif action == 2: force[0] = -power
            elif action == 3: force[0] = power

            s["vel"] += force
            s["vel"] *= s["friction"]
            s["pos"] += s["vel"]

            # NEW: Calculate reward for this specific action
            total_step_reward += self.get_reward(i)

        return total_step_reward

    def draw(self, surface, total_reward, step):
        super().draw(surface)
        # Overlay the Reward so we can see it in the video!
        font = pygame.font.SysFont('Arial', 24)
        img = font.render(f"Step: {step} | Cumulative Reward: {int(total_reward)}", True, (255, 255, 255))
        surface.blit(img, (20, 20))

In [15]:
# Running with Rewards

def run_reward_experiment(world_class, seconds=10):
    pygame.init()
    width, height = 640, 480
    surface = pygame.Surface((width, height))
    world = world_class(width, height)
    recorder = VideoRecorder(width=width, height=height)

    recorder.start()
    cumulative_reward = 0

    for t in range(30 * seconds):
        # Update now returns the reward for that step
        step_reward = world.update(t)
        cumulative_reward += step_reward

        # Draw with the reward overlay
        world.draw(surface, cumulative_reward, t)
        recorder.record_frame(surface)

    final_file = recorder.stop()
    pygame.quit()
    display(Video(final_file, embed=True, width=600))



In [19]:
run_reward_experiment(FrogSnakeWorldV3R)

#Chronobooks <br>
Three science fiction novels by Prithwis Mukerjee. A dystopian Earth. A technocratic society managed by artificial intelligence. Escape and epiphany on Mars. Can man and machine, carbon and silicon explore and escape into other dimensions of existence? An Indic perspective rooted in Advaita Vedanta and the Divine Feminine.  [More information](http://bit.ly/chrono3) <br>
![alt text](https://blogger.googleusercontent.com/img/a/AVvXsEjsZufX_KYaLwAnJP6bUxvDg5RSPn6r8HIZe749nLWX3RuwyshrYEAUpdw03a9WIWRdnzA9epwJOE05eDJ0Ad7kGyfWiUrC2vNuOskb2jA-e8aOZSx8YqzT8mfZi3E4X1Rz3qlEAiv-aTxlCM976BEeTjx4J64ctY3C_FoV4v9aY_U23F8xRqI5Eg=s1600)